## Navigate to the Correct Directory

The following code navigates to the dataprocessing directory.

In [2]:
cd ../dataprocessing

/usr/local/google/home/carverforbes/activity-recognition/location/lbs/activity/audioset/dataprocessing


## Call the Import Statements

The following code imports the necessary code to run the code in the rest of this notebook.

In [3]:
# import statements
import audio_processing as ap

import functools
import os
import sys
from absl import logging

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import sklearn

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

Ran the import statements.


## Feature Extraction
Configure the following parameters to extract the desired features from a specified csv file to a specific destination directory.

In [4]:
# set logging to print logging.INFO logs
logging.set_verbosity(logging.INFO)

In [5]:
# arguments for audio_processing
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
filename = 'gunshot_30_70'
labels = ['Gunshot, gunfire']
available_features = ['chroma_stft',
                       'chroma_cqt',
                       'chroma_cens',
                       'melspectrogram',
                       'mfcc',
                       'rms',
                       'spectral_centroid',
                       'spectral_bandwidth',
                       'spectral_contrast',
                       'spectral_flatness',
                       'spectral_rolloff',
                       'poly_features',
                       'tonnetz',
                       'zero_crossing_rate']
features_to_extract = ['melspectrogram']
last_features = None
redo = False

In [ ]:
def dataframe_to_csv(dataframe, dest_path):
    df = dataframe.copy()
    for column in df.columns:   
        for i in range(df[column].size):
            if isinstance(df[column][i], np.ndarray):
                df[column][i] = df[column][i].tolist()
    df.to_csv(path, index=False, header=True)

In [ ]:
def csv_to_dataframe(csv_path):
    df = pd.read_csv(csv_path)
    for column in df.columns:
        for i in range(df[column].size):
            if isinstance(df[column][i], str):
                temp = df[column][i][1:-1]
                temp = temp.replace(',', ' ').split(']')
                new_list = []
                for item in temp:
                    if item == '':
                        continue
                    item = item.replace('[', '').strip().split()
                    item = [float(num) for num in item]
                    arr = np.array(item)
                    new_list.append(arr)
                new_arr = np.array(new_list)
                df[column][i] = new_arr
    return df

In [7]:
# Import the dataset as a pandas Dataframe object.
df = ap.output_df(src_dir, dest_dir, filename, labels, features_to_extract, False)

INFO:absl:The set has 23728 examples
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:a

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

...

INFO:absl:(20450, 13467.993908)
INFO:absl:extracted features
INFO:absl:(20451, 13468.693222)
INFO:absl:extracted features
INFO:absl:(20452, 13469.314102)
INFO:absl:extracted features
INFO:absl:(20453, 13470.018999)
INFO:absl:extracted features
INFO:absl:(20454, 13470.67968)
INFO:absl:extracted features
INFO:absl:(20455, 13471.274555)
INFO:absl:extracted features
INFO:absl:(20456, 13471.904196)
INFO:absl:extracted features
INFO:absl:(20457, 13472.501182)
INFO:absl:extracted features
INFO:absl:(20458, 13473.063223)
INFO:absl:extracted features
INFO:absl:(20459, 13473.620473)
INFO:absl:extracted features
INFO:absl:(20460, 13474.151297)
INFO:absl:extracted features
INFO:absl:(20461, 13474.759333)
INFO:absl:extracted features
INFO:absl:(20462, 13475.358647)
INFO:absl:extracted features
INFO:absl:(20463, 13476.052016)
INFO:absl:extracted features
INFO:absl:(20464, 13476.658811)
INFO:absl:extracted features
INFO:absl:(20465, 13477.371631)
INFO:absl:extracted features
INFO:absl:(20466, 13477.9

INFO:absl:(20585, 13553.398926)
INFO:absl:extracted features
INFO:absl:(20586, 13553.991536)
INFO:absl:extracted features
INFO:absl:(20587, 13554.672902)
INFO:absl:extracted features
INFO:absl:(20588, 13555.361108)
INFO:absl:extracted features
INFO:absl:(20589, 13555.960977)
INFO:absl:extracted features
INFO:absl:(20590, 13556.54253)
INFO:absl:extracted features
INFO:absl:(20591, 13557.178066)
INFO:absl:extracted features
INFO:absl:(20592, 13557.86596)
INFO:absl:extracted features
INFO:absl:(20593, 13558.42524)
INFO:absl:extracted features
INFO:absl:(20594, 13559.009851)
INFO:absl:extracted features
INFO:absl:(20595, 13559.616702)
INFO:absl:extracted features
INFO:absl:(20596, 13560.36749)
INFO:absl:extracted features
INFO:absl:(20597, 13560.943717)
INFO:absl:extracted features
INFO:absl:(20598, 13561.51035)
INFO:absl:extracted features
INFO:absl:(20599, 13562.12319)
INFO:absl:extracted features
INFO:absl:(20600, 13562.67679)
INFO:absl:extracted features
INFO:absl:(20601, 13563.274695)

INFO:absl:(20720, 13636.748913)
INFO:absl:extracted features
INFO:absl:(20721, 13637.387707)
INFO:absl:extracted features
INFO:absl:(20722, 13638.062119)
INFO:absl:extracted features
INFO:absl:(20723, 13638.676039)
INFO:absl:extracted features
INFO:absl:(20724, 13639.670787)
INFO:absl:extracted features
INFO:absl:(20725, 13640.319321)
INFO:absl:extracted features
INFO:absl:(20726, 13640.938959)
INFO:absl:extracted features
INFO:absl:(20727, 13641.528626)
INFO:absl:extracted features
INFO:absl:(20728, 13642.173677)
INFO:absl:extracted features
INFO:absl:(20729, 13642.747791)
INFO:absl:extracted features
INFO:absl:(20730, 13643.495313)
INFO:absl:extracted features
INFO:absl:(20731, 13644.112324)
INFO:absl:extracted features
INFO:absl:(20732, 13644.71172)
INFO:absl:extracted features
INFO:absl:(20733, 13645.291468)
INFO:absl:extracted features
INFO:absl:(20734, 13645.883773)
INFO:absl:extracted features
INFO:absl:(20735, 13646.51028)
INFO:absl:extracted features
INFO:absl:(20736, 13647.14

INFO:absl:extracted features
INFO:absl:(20855, 13720.603507)
INFO:absl:extracted features
INFO:absl:(20856, 13721.237943)
INFO:absl:extracted features
INFO:absl:(20857, 13721.830139)
INFO:absl:extracted features
INFO:absl:(20858, 13722.483871)
INFO:absl:extracted features
INFO:absl:(20859, 13723.013062)
INFO:absl:extracted features
INFO:absl:(20860, 13723.563099)
INFO:absl:extracted features
INFO:absl:(20861, 13724.169869)
INFO:absl:extracted features
INFO:absl:(20862, 13724.808877)
INFO:absl:extracted features
INFO:absl:(20863, 13725.477598)
INFO:absl:extracted features
INFO:absl:(20864, 13726.120094)
INFO:absl:extracted features
INFO:absl:(20865, 13726.73172)
INFO:absl:extracted features
INFO:absl:(20866, 13727.413351)
INFO:absl:extracted features
INFO:absl:(20867, 13728.017548)
INFO:absl:extracted features
INFO:absl:(20868, 13728.716631)
INFO:absl:extracted features
INFO:absl:(20869, 13729.389526)
INFO:absl:extracted features
INFO:absl:(20870, 13730.002141)
INFO:absl:extracted featu

INFO:absl:extracted features
INFO:absl:(20990, 13805.514725)
INFO:absl:extracted features
INFO:absl:(20991, 13806.156154)
INFO:absl:extracted features
INFO:absl:(20992, 13806.737085)
INFO:absl:extracted features
INFO:absl:(20993, 13807.581426)
INFO:absl:extracted features
INFO:absl:(20994, 13808.333124)
INFO:absl:extracted features
INFO:absl:(20995, 13808.87669)
INFO:absl:extracted features
INFO:absl:(20996, 13809.463559)
INFO:absl:extracted features
INFO:absl:(20997, 13810.014515)
INFO:absl:extracted features
INFO:absl:(20998, 13810.760614)
INFO:absl:extracted features
INFO:absl:(20999, 13811.38801)
INFO:absl:extracted features
INFO:absl:(21000, 13811.998581)
INFO:absl:extracted features
INFO:absl:(21001, 13812.576812)
INFO:absl:extracted features
INFO:absl:(21002, 13813.151731)
INFO:absl:extracted features
INFO:absl:(21003, 13813.697945)
INFO:absl:extracted features
INFO:absl:(21004, 13814.354964)
INFO:absl:extracted features
INFO:absl:(21005, 13814.94714)
INFO:absl:extracted feature

In [6]:
# print the first 5 rows of the dataframe.
df.head()

NameError: name 'df' is not defined

## Data Preprocessing

In [28]:
# copy the dataframe object to perform preprocessing without modifying the original extracted features.
train_df_mel = df.copy()
train_df_mel.head()

,label,melspectrogram
0,1,"[[3.4128592, 2.5363638, 2.2952628, 1.9726387, ..."
1,1,"[[0.009717191, 0.0028681338, 0.0013111196, 0.0..."
2,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1,"[[0.17820564, 0.09990731, 0.043435965, 0.10587..."
4,0,"[[0.00096251303, 0.0006070902, 0.0003984337, 0..."


In [29]:
# Shuffle the dataset/dataframe.
train_df_mel = train_df_rms.reindex(
    np.random.permutation(train_df_rms.index))
train_df_mel.head()

,label,melspectrogram
15381,0,"[[2.2586117e-07, 6.895915e-07, 1.9162449e-06, ..."
19380,0,"[[4.4546945e-05, 6.677826e-05, 7.545271e-05, 0..."
19877,0,"[[0.008395814, 0.0021436212, 0.000116237854, 0..."
6697,0,"[[6.819979e-05, 3.9538285e-05, 3.6845497e-05, ..."
7571,0,"[[15.440015, 3.7069643, 0.20693178, 0.17341408..."


In [31]:
# temp bug fix for rows with None features
# and rows with different number of elements per frame
bad_rows = []
print(train_df_mel.size / 2)
# print(train_df_rms.rms[0])
for i in train_df_mel.index:
    if train_df_mel.melspectrogram[i] is None:
        bad_rows.append(i)
        continue
    if len(train_df_mel.melspectrogram[i]) != 128:
        bad_rows.append(i)
    if len(train_df_mel.melspectrogram[i][0]) != 431:
        bad_rows.append(i)
# print(bad_rows)
print(len(bad_rows))
train_df_mel = train_df_mel.drop(bad_rows)
train_df_mel.head(10000)

19458.0
0


,label,melspectrogram
15381,0,"[[2.2586117e-07, 6.895915e-07, 1.9162449e-06, ..."
19380,0,"[[4.4546945e-05, 6.677826e-05, 7.545271e-05, 0..."
19877,0,"[[0.008395814, 0.0021436212, 0.000116237854, 0..."
6697,0,"[[6.819979e-05, 3.9538285e-05, 3.6845497e-05, ..."
7571,0,"[[15.440015, 3.7069643, 0.20693178, 0.17341408..."
...,...,...
4286,1,"[[0.48813453, 0.47872972, 0.4568061, 0.4965630..."
8287,0,"[[2.4993176, 2.78743, 2.366105, 1.5476923, 2.1..."
15170,0,"[[0.051547613, 0.049831, 0.040460356, 0.005196..."
15884,0,"[[0.110663116, 0.029383905, 0.0023546196, 0.00..."


Enter your data preprocessing here.

In [1]:
X = np.array(train_df_mel.melspectrogram.tolist(), dtype=object, ndmin=3)
y = np.array(train_df_mel.label.tolist())

NameError: name 'np' is not defined

In [ ]:
# Flatten melspectrogram's (128, 431) shaped features 
temp_x = []
for arr in X:
    arr = arr.flatten()
    temp_x.append(arr)
X = np.array(temp_x, dtype=object)

In [ ]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
# Convert arrays of objects to arrays of floats.
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
y_train = K.cast_to_floatx(y_train)
x_val = K.cast_to_floatx(x_val)
y_val = K.cast_to_floatx(y_val)

## Building and Training Neural Network

In [ ]:
# Define the plotting function.
def plot_curve(epochs, hist, list_of_metrics, path, filename, list_of_hyperparameters):
    """Plot a curve of one or more classification metrics vs. epoch and save it to path."""  
    # list_of_metrics should be one of the names shown in:
    # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)

    plt.legend()
    
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
            
    path = os.path.join(path, filename)
    
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
        
    list_of_hyperparameters_temp = [str(item) for item in list_of_hyperparameters]
    list_of_metrics_temp = [item if isinstance(item, str) else str(item.name) for item in list_of_metrics]
    filename = '_'.join(list_of_metrics_temp) + '_' + '_'.join(list_of_hyperparameters_temp)
    path = os.path.join(path, filename + '.png')
    plt.savefig(path, bbox_inches='tight')
    
    return plt


print("Defined the plot_curve function.")

## Train 1
Neural Network Type:
* Input Layer: (Number of Nodes)
* Hidden Layer 1:
* Hidden Layer 2: 
* Ouput Layer: 

Hyper-parameters:
* Loss Function: 
* Activation Function: 
* Optimizer Function: 
* Learning Rate: 
* Epochs: 
* Batch_Size: 
* Classification Threshold: 
* Regularization: 
* Regularization Lambda: 

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, optimizer, regularization):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of ___ nodes
    model.add(tf.keras.layers.Dense(units=, input_shape=(,)))
              
    # Implement ___ regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))
    
    # Implement ___ regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

In [ ]:
# The following variables are the hyperparameters.
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 10
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot)
plot_curve(epochs, hist, ['loss', 'val_loss'])

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

In [ ]:
# The following variables are the hyperparameters.
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 10
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"

METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]